### Template strings

- Python 3.14 introduces t-strings, which you can treat as an extension of the usual f-string

- You can see that a tstring returns a `Template` type, while an fstring just returns a string

In [4]:
name = 'yj'
tstring = t"hello {name}"
fstring = f"hello {name}"

type(tstring), type(fstring)

(string.templatelib.Template, str)

- The great thing about the `Template` is that you can sanitize inputs very easily, which avoids issues like SQL injection

- Since th return value is a `Template`, you can easily cast it to anything else. For example, can return a `Prompt` in Langgraph

In [9]:
from string.templatelib import Interpolation

amended_string=[]
for element in tstring:
    if isinstance(element, Interpolation):
        amended_string.append(str(element.value).upper())
    else:
        amended_string.append(element)

''.join(amended_string)


'hello YJ'

### Deferred Evaluation of Annocation, and `annotationlib`

- In previous iterations of Python, type hints are evaluated in order at runtime. So if you had functions with some class defined in the signature, with the class defined later in your code, you end up needing to use strings as the type annotation instead, or you get a runtime error

- Now, type evaluation is deferred, so you no longer get an error

In [ ]:
from dataclasses import dataclass

## No longer get runtime errors for Input1 and Output1
def somefunc(input1: Input1) -> Output1:
    pass

@dataclass
class Input1:
    attr1: str
    attr2: float

@dataclass
class Output1:
    output1: float
    output2: list[float]


### Multiple Interpreters + InterpreterPoolExecutor

- Previously, multiprocessing in python requires spawning a bunch of different processes

- While each process has their own GIL, making parallel processing feasible, this is problematic because different processes do not share memory. So if you need to share data, SERDE is needed

- In 3.14, multiprocessing is now enabled within the same process via sub-interpreters. Shared data is possible, but it also means that if the main interpreter crashes, the subinterpreters also dies

In [14]:
from concurrent import interpreters

interpreter = interpreters.create()
interpreter.exec('''print("hello from interpreter")''')

def square(n):
    return n*n

print(interpreter.call(square, 5))

25hello from interpreter



In [17]:
from concurrent.futures import InterpreterPoolExecutor, ThreadPoolExecutor, as_completed
import time
import math

def compute_factorial_concurrent():
    start=time.perf_counter()
    res = []

    with InterpreterPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(math.factorial, i) for i in range(10000,15000)]

        for future in as_completed(futures):
            res.append(future.result())

    end=time.perf_counter()
    return end-start, res

def compute_factorial_threaded():
    start=time.perf_counter()
    res = []

    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(math.factorial, i) for i in range(10000,15000)]

        for future in as_completed(futures):
            res.append(future.result())

    end=time.perf_counter()
    return end-start, res

def compute_factorial_seqeuntial():
    start=time.perf_counter()
    res = [math.factorial(i) for i in range(10000, 15000)]
    end=time.perf_counter()
    return end-start, res


ctime, cres = compute_factorial_concurrent()
ttime, tres = compute_factorial_threaded()
stime, sres = compute_factorial_seqeuntial()

print(ctime, ttime, stime)

5.419310166995274 22.79251599998679 20.940022791997762


### Free-threaded Python

- Not only can we have sub-interpreters to do multiprocessing, 3.14 allows you to compile python without the GIL. This is known as free threaded python

- There is a difference between free-threaded multithreading vs multi processing via sub-interpreters
    - Free-threaded multithreading:
        - All threads can run python bytecode at the same time
        - All threads share memory
        - BUT you are responsible for handling race conditions vs locks and mutex
    - Multiprocessing with sub-interpreter
        - Every subinterpreter runs in the same process, and can execute code in parallel 
        - BUT Every subinterpreter is more or less isolated, and sharing memory will require you to used some specific mechanisms (queues, shared memory etc)

- If you rerun the benchmark in previous section with free threaded python, the time taken for subinterpreter and multithreading is ~the same

### asyncio CLI

- You can use a CLI tool to look at asyncio tasks

- Assume you have a python process running and awaiting; you can use the call below to see the asyncio task status
    - `sudo .venv/bin/python  -m  asyncio ps 12345`
    - `sudo .venv/bin/python  -m  asyncio pstree 12345` 

- The process value can be found by running `os.getpid()`

### Compression algorithms combined into 1 package

In [19]:
from compression import zlib, bz2, lzma, gzip

### 